In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
urola_raw_path = r'/opt/dora/Dora/NAIR/IA4FLOOD/Urola/data/'
urola_data = os.listdir(urola_raw_path)
urola_data.sort()
print(urola_data)

result = []
for item in urola_data:
    name_river = item.split('_')
    if len(name_river) > 1:
        name_river = item.split('_')[0] + '_' + item.split('_')[1]
        if name_river not in result:
            result.append(name_river)
    else:
        name_river = item.split('_')[0]
        if name_river not in result:
            result.append(name_river)

print(result)

['Aitzu-Urola_Estacion_Aforos_01oct1998_30sep1999.csv', 'Aitzu-Urola_Estacion_Aforos_01oct1999_30sep2000.csv', 'Aitzu-Urola_Estacion_Aforos_01oct2000_30sep2001.csv', 'Aitzu-Urola_Estacion_Aforos_01oct2001_30sep2002.csv', 'Aitzu-Urola_Estacion_Aforos_01oct2002_30sep2003.csv', 'Aitzu-Urola_Estacion_Aforos_01oct2003_30sep2004.csv', 'Aitzu-Urola_Estacion_Aforos_01oct2004_30sep2005.csv', 'Aitzu-Urola_Estacion_Aforos_01oct2005_30sep2006.csv', 'Aitzu-Urola_Estacion_Aforos_01oct2006_30sep2007.csv', 'Aitzu-Urola_Estacion_Aforos_01oct2007_30sep2008.csv', 'Aitzu-Urola_Estacion_Aforos_01oct2008_30sep2009.csv', 'Aitzu-Urola_Estacion_Aforos_01oct2009_30sep2010.csv', 'Aitzu-Urola_Estacion_Aforos_01oct2010_30sep2011.csv', 'Aitzu-Urola_Estacion_Aforos_01oct2011_30sep2012.csv', 'Aitzu-Urola_Estacion_Aforos_01oct2012_30sep2013.csv', 'Aitzu-Urola_Estacion_Aforos_01oct2013_30sep2014.csv', 'Aitzu-Urola_Estacion_Aforos_01oct2014_30sep2015.csv', 'Aitzu-Urola_Estacion_Aforos_01oct2015_30sep2016.csv', 'Aitzu-Ur

In [3]:
# Crear un rango de fechas desde el 1 de enero de 2000 hasta el 31 de diciembre de 2024, cada 60 minutos
fecha_inicio = '1999-01-01'
fecha_fin = '2024-12-31 23:59'
frecuencia = 'H'  # '1H'

# Usando date_range para generar las fechas
rango_fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq=frecuencia)

# Crear un DataFrame con las fechas
df_fechas = pd.DataFrame(rango_fechas, columns=['Fecha'])
df_fechas['Fecha'] = pd.to_datetime(df_fechas['Fecha'])

In [4]:
def grafica_dato_faltant (df):
    # Graficar los datos faltantes
    plt.figure(figsize=(5, 10))
    sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
    plt.title('Datos faltantes en el dataset')
    plt.show()

def read_database(database, numer_file1, number_file2, estacion):
    urola_aux1 =pd.DataFrame()
    for x in urola_data[numer_file1:number_file2]:
        print(x)
        urola = pd.read_csv(database + x, sep=';', dayfirst=True, encoding='latin1')
        if estacion == 'Estacion':
            urola_aux = urola[['FECHA', 'HORA (GMT)', 'LLUVIA (mm/m2)', 'NIVEL_RIO (m)', 'CAUDAL_RIO (m3/sg)']]
        elif estacion == 'Pluviometro':
            urola_aux = urola[['FECHA', 'HORA (GMT)', 'LLUVIA (mm/m2)']]
        urola_aux.loc[:,'FECHA'] = pd.to_datetime(urola_aux['FECHA'] + ' ' + urola_aux['HORA (GMT)'], format='%d-%m-%Y %H:%M', errors='coerce')
        urola_aux = urola_aux.drop(columns=['HORA (GMT)'])
        replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
        urola_aux.replace(replace_dict, inplace=True)  
        urola_aux1 = pd.concat([urola_aux1, urola_aux]) 
    urola_aux1['FECHA'] = pd.to_datetime(urola_aux1['FECHA'], format='%d-%m-%Y %H:%M', errors='coerce')
    return urola_aux1, x

In [5]:
# df_list = []
# urola_aux1 =pd.DataFrame()
# for x in urola_data[0:25]:
#     print(x)
#     urola = pd.read_csv(urola_raw_path + x, sep=';', dayfirst=True, encoding='latin1')
#     urola_aux = urola[['FECHA', 'HORA (GMT)', 'LLUVIA (mm/m2)', 'NIVEL_RIO (m)', 'CAUDAL_RIO (m3/sg)']]
#     urola_aux.loc[:,'FECHA'] = pd.to_datetime(urola_aux['FECHA'] + ' ' + urola_aux['HORA (GMT)'], format='%d-%m-%Y %H:%M', errors='coerce')
#     urola_aux = urola_aux.drop(columns=['HORA (GMT)'])
#     urola_aux1 = pd.concat([urola_aux1, urola_aux]) 

urola_aux1, x = read_database(urola_raw_path, 0,25, 'Estacion')
df_aitzu_urola = df_fechas.copy()
df_aitzu_urola = pd.merge(df_aitzu_urola, urola_aux1, left_on='Fecha', right_on='FECHA', how='inner')
df_aitzu_urola = df_aitzu_urola.drop(columns=['FECHA'])
df_aitzu_urola.rename(columns={'LLUVIA (mm/m2)': 'LLUVIA_' + x.split('_')[0], 'NIVEL_RIO (m)': 'NIVEL_RIO_' + x.split('_')[0], 'CAUDAL_RIO (m3/sg)': 'CAUDAL_RIO_' + x.split('_')[0]}, inplace=True)
replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
df_aitzu_urola.replace(replace_dict, inplace=True)


Aitzu-Urola_Estacion_Aforos_01oct1998_30sep1999.csv
Aitzu-Urola_Estacion_Aforos_01oct1999_30sep2000.csv
Aitzu-Urola_Estacion_Aforos_01oct2000_30sep2001.csv
Aitzu-Urola_Estacion_Aforos_01oct2001_30sep2002.csv
Aitzu-Urola_Estacion_Aforos_01oct2002_30sep2003.csv
Aitzu-Urola_Estacion_Aforos_01oct2003_30sep2004.csv
Aitzu-Urola_Estacion_Aforos_01oct2004_30sep2005.csv
Aitzu-Urola_Estacion_Aforos_01oct2005_30sep2006.csv
Aitzu-Urola_Estacion_Aforos_01oct2006_30sep2007.csv
Aitzu-Urola_Estacion_Aforos_01oct2007_30sep2008.csv
Aitzu-Urola_Estacion_Aforos_01oct2008_30sep2009.csv
Aitzu-Urola_Estacion_Aforos_01oct2009_30sep2010.csv
Aitzu-Urola_Estacion_Aforos_01oct2010_30sep2011.csv
Aitzu-Urola_Estacion_Aforos_01oct2011_30sep2012.csv
Aitzu-Urola_Estacion_Aforos_01oct2012_30sep2013.csv
Aitzu-Urola_Estacion_Aforos_01oct2013_30sep2014.csv
Aitzu-Urola_Estacion_Aforos_01oct2014_30sep2015.csv
Aitzu-Urola_Estacion_Aforos_01oct2015_30sep2016.csv
Aitzu-Urola_Estacion_Aforos_01oct2016_30sep2017.csv
Aitzu-Urola_

In [6]:
urola_aux1, x = read_database(urola_raw_path, 25,50, 'Estacion')
df_aitzu = df_fechas.copy()
df_aitzu = pd.merge(df_aitzu, urola_aux1, left_on='Fecha', right_on='FECHA', how='inner')
df_aitzu = df_aitzu.drop(columns=['FECHA'])
df_aitzu.rename(columns={'LLUVIA (mm/m2)': 'LLUVIA_' + x.split('_')[0], 'NIVEL_RIO (m)': 'NIVEL_RIO_' + x.split('_')[0], 'CAUDAL_RIO (m3/sg)': 'CAUDAL_RIO_' + x.split('_')[0]},inplace = True)
replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
df_aitzu.replace(replace_dict, inplace=True)

Aitzu_Estacion_Aforos_01oct1998_30sep1999.csv
Aitzu_Estacion_Aforos_01oct1999_30sep2000.csv
Aitzu_Estacion_Aforos_01oct2000_30sep2001.csv
Aitzu_Estacion_Aforos_01oct2001_30sep2002.csv
Aitzu_Estacion_Aforos_01oct2002_30sep2003.csv
Aitzu_Estacion_Aforos_01oct2003_30sep2004.csv
Aitzu_Estacion_Aforos_01oct2004_30sep2005.csv
Aitzu_Estacion_Aforos_01oct2005_30sep2006.csv
Aitzu_Estacion_Aforos_01oct2006_30sep2007.csv
Aitzu_Estacion_Aforos_01oct2007_30sep2008.csv
Aitzu_Estacion_Aforos_01oct2008_30sep2009.csv
Aitzu_Estacion_Aforos_01oct2009_30sep2010.csv
Aitzu_Estacion_Aforos_01oct2010_30sep2011.csv
Aitzu_Estacion_Aforos_01oct2011_30sep2012.csv
Aitzu_Estacion_Aforos_01oct2012_30sep2013.csv
Aitzu_Estacion_Aforos_01oct2013_30sep2014.csv
Aitzu_Estacion_Aforos_01oct2014_30sep2015.csv
Aitzu_Estacion_Aforos_01oct2015_30sep2016.csv
Aitzu_Estacion_Aforos_01oct2016_30sep2017.csv
Aitzu_Estacion_Aforos_01oct2017_30sep2018.csv
Aitzu_Estacion_Aforos_01oct2018_30sep2019.csv
Aitzu_Estacion_Aforos_01oct2019_30

In [7]:
urola_aux1, x = read_database(urola_raw_path, 50,78, 'Estacion')
    
df_aizarnazabal = df_fechas.copy()
df_aizarnazabal = pd.merge(df_aizarnazabal, urola_aux1, left_on='Fecha', right_on='FECHA', how='inner')
df_aizarnazabal = df_aizarnazabal.drop(columns=['FECHA'])
df_aizarnazabal.rename(columns={'LLUVIA (mm/m2)': 'LLUVIA_' + x.split('_')[0], 'NIVEL_RIO (m)': 'NIVEL_RIO_' + x.split('_')[0], 'CAUDAL_RIO (m3/sg)': 'CAUDAL_RIO_' + x.split('_')[0]},inplace = True)
replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
df_aizarnazabal.replace(replace_dict, inplace=True)

Aizarnazabal_Estacion_Aforos_01oct1995_30sep1996.csv
Aizarnazabal_Estacion_Aforos_01oct1996_30sep1997.csv
Aizarnazabal_Estacion_Aforos_01oct1997_30sep1998.csv
Aizarnazabal_Estacion_Aforos_01oct1998_30sep1999.csv
Aizarnazabal_Estacion_Aforos_01oct1999_30sep2000.csv
Aizarnazabal_Estacion_Aforos_01oct2000_30sep2001.csv
Aizarnazabal_Estacion_Aforos_01oct2001_30sep2002.csv
Aizarnazabal_Estacion_Aforos_01oct2002_30sep2003.csv
Aizarnazabal_Estacion_Aforos_01oct2003_30sep2004.csv
Aizarnazabal_Estacion_Aforos_01oct2004_30sep2005.csv
Aizarnazabal_Estacion_Aforos_01oct2005_30sep2006.csv
Aizarnazabal_Estacion_Aforos_01oct2006_30sep2007.csv
Aizarnazabal_Estacion_Aforos_01oct2007_30sep2008.csv
Aizarnazabal_Estacion_Aforos_01oct2008_30sep2009.csv
Aizarnazabal_Estacion_Aforos_01oct2009_30sep2010.csv
Aizarnazabal_Estacion_Aforos_01oct2010_30sep2011.csv
Aizarnazabal_Estacion_Aforos_01oct2011_30sep2012.csv
Aizarnazabal_Estacion_Aforos_01oct2012_30sep2013.csv
Aizarnazabal_Estacion_Aforos_01oct2013_30sep20

In [8]:
urola_aux1, x = read_database(urola_raw_path, 422,444, 'Estacion')
df_Matxinbenta_Estacion = df_fechas.copy()
df_Matxinbenta_Estacion = pd.merge(df_Matxinbenta_Estacion, urola_aux1, left_on='Fecha', right_on='FECHA', how='inner')
df_Matxinbenta_Estacion = df_Matxinbenta_Estacion.drop(columns=['FECHA'])
df_Matxinbenta_Estacion.rename(columns={'LLUVIA (mm/m2)': 'LLUVIA_' + x.split('_')[0], 'NIVEL_RIO (m)': 'NIVEL_RIO_' + x.split('_')[0], 'CAUDAL_RIO (m3/sg)': 'CAUDAL_RIO_' + x.split('_')[0]},inplace = True)
replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
df_Matxinbenta_Estacion.replace(replace_dict, inplace=True)

Matxinbenta_Estacion_Aforos_01oct2001_30sep2002.csv
Matxinbenta_Estacion_Aforos_01oct2002_30sep2003.csv
Matxinbenta_Estacion_Aforos_01oct2003_30sep2004.csv
Matxinbenta_Estacion_Aforos_01oct2004_30sep2005.csv
Matxinbenta_Estacion_Aforos_01oct2005_30sep2006.csv
Matxinbenta_Estacion_Aforos_01oct2006_30sep2007.csv
Matxinbenta_Estacion_Aforos_01oct2007_30sep2008.csv
Matxinbenta_Estacion_Aforos_01oct2008_30sep2009.csv
Matxinbenta_Estacion_Aforos_01oct2009_30sep2010.csv
Matxinbenta_Estacion_Aforos_01oct2010_30sep2011.csv
Matxinbenta_Estacion_Aforos_01oct2011_30sep2012.csv
Matxinbenta_Estacion_Aforos_01oct2012_30sep2013.csv
Matxinbenta_Estacion_Aforos_01oct2013_30sep2014.csv
Matxinbenta_Estacion_Aforos_01oct2014_30sep2015.csv
Matxinbenta_Estacion_Aforos_01oct2015_30sep2016.csv
Matxinbenta_Estacion_Aforos_01oct2016_30sep2017.csv
Matxinbenta_Estacion_Aforos_01oct2017_30sep2018.csv
Matxinbenta_Estacion_Aforos_01oct2018_30sep2019.csv
Matxinbenta_Estacion_Aforos_01oct2019_30sep2020.csv
Matxinbenta_

In [9]:
urola_aux1, x = read_database(urola_raw_path, 290,323, 'Estacion')
df_Ibaieder_Estacion= df_fechas.copy()
df_Ibaieder_Estacion = pd.merge(df_Ibaieder_Estacion, urola_aux1, left_on='Fecha', right_on='FECHA', how='inner')
df_Ibaieder_Estacion = df_Ibaieder_Estacion.drop(columns=['FECHA'])
df_Ibaieder_Estacion.rename(columns={'LLUVIA (mm/m2)': 'LLUVIA_' + x.split('_')[0] ,'NIVEL_RIO (m)': 'NIVEL_RIO_' + x.split('_')[0], 'CAUDAL_RIO (m3/sg)': 'CAUDAL_RIO_' + x.split('_')[0]},inplace = True)
replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
df_Ibaieder_Estacion.replace('-1', np.nan, inplace=True)

Ibaieder_Estacion_Aforos_01oct1990_30sep1991.csv
Ibaieder_Estacion_Aforos_01oct1991_30sep1992.csv
Ibaieder_Estacion_Aforos_01oct1992_30sep1993.csv
Ibaieder_Estacion_Aforos_01oct1993_30sep1994.csv
Ibaieder_Estacion_Aforos_01oct1994_30sep1995.csv
Ibaieder_Estacion_Aforos_01oct1995_30sep1996.csv
Ibaieder_Estacion_Aforos_01oct1996_30sep1997.csv
Ibaieder_Estacion_Aforos_01oct1997_30sep1998.csv
Ibaieder_Estacion_Aforos_01oct1998_30sep1999.csv
Ibaieder_Estacion_Aforos_01oct1999_30sep2000.csv
Ibaieder_Estacion_Aforos_01oct2000_30sep2001.csv
Ibaieder_Estacion_Aforos_01oct2001_30sep2002.csv
Ibaieder_Estacion_Aforos_01oct2002_30sep2003.csv
Ibaieder_Estacion_Aforos_01oct2003_30sep2004.csv
Ibaieder_Estacion_Aforos_01oct2004_30sep2005.csv
Ibaieder_Estacion_Aforos_01oct2005_30sep2006.csv
Ibaieder_Estacion_Aforos_01oct2006_30sep2007.csv
Ibaieder_Estacion_Aforos_01oct2007_30sep2008.csv
Ibaieder_Estacion_Aforos_01oct2008_30sep2009.csv
Ibaieder_Estacion_Aforos_01oct2009_30sep2010.csv
Ibaieder_Estacion_Af

In [10]:
urola_aux1, x = read_database(urola_raw_path, 168,200, 'Estacion')
df_barrendiola_estacion = df_fechas.copy()
df_barrendiola_estacion = pd.merge(df_barrendiola_estacion, urola_aux1, left_on='Fecha', right_on='FECHA', how='inner')
df_barrendiola_estacion = df_barrendiola_estacion.drop(columns=['FECHA'])
df_barrendiola_estacion.rename(columns={'LLUVIA (mm/m2)': 'LLUVIA_' + x.split('_')[0], 'NIVEL_RIO (m)': 'NIVEL_RIO_' + x.split('_')[0], 'CAUDAL_RIO (m3/sg)': 'CAUDAL_RIO_' + x.split('_')[0]},inplace = True)
replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
df_barrendiola_estacion.replace(replace_dict, inplace=True)

Barrendiola_Estacion_AForos_01oct1993_30sep1994.csv
Barrendiola_Estacion_Aforos_01oct1990_30sep1991.csv
Barrendiola_Estacion_Aforos_01oct1991_30sep1992.csv
Barrendiola_Estacion_Aforos_01oct1992_30sep1993.csv
Barrendiola_Estacion_Aforos_01oct1994_30sep1995.csv
Barrendiola_Estacion_Aforos_01oct1995_30sep1996.csv
Barrendiola_Estacion_Aforos_01oct1996_30sep1997.csv
Barrendiola_Estacion_Aforos_01oct1997_30sep1998.csv
Barrendiola_Estacion_Aforos_01oct1998_30sep1999.csv
Barrendiola_Estacion_Aforos_01oct1999_30sep2000.csv
Barrendiola_Estacion_Aforos_01oct2000_30sep2001.csv
Barrendiola_Estacion_Aforos_01oct2001_30sep2002.csv
Barrendiola_Estacion_Aforos_01oct2002_30sep2003.csv
Barrendiola_Estacion_Aforos_01oct2003_30sep2004.csv
Barrendiola_Estacion_Aforos_01oct2004_30sep2005.csv
Barrendiola_Estacion_Aforos_01oct2005_30sep2006.csv
Barrendiola_Estacion_Aforos_01oct2007_30sep2008.csv
Barrendiola_Estacion_Aforos_01oct2008_30sep2009.csv
Barrendiola_Estacion_Aforos_01oct2009_30sep2010.csv
Barrendiola_

In [11]:
urola_aux1, x = read_database(urola_raw_path, 78,83, 'Pluviometro')
df_antzuola_pluviometro = df_fechas.copy()
df_antzuola_pluviometro = pd.merge(df_antzuola_pluviometro, urola_aux1, left_on='Fecha', right_on='FECHA', how='inner')
df_antzuola_pluviometro = df_antzuola_pluviometro.drop(columns=['FECHA'])
df_antzuola_pluviometro.rename(columns={'LLUVIA (mm/m2)': 'LLUVIA_' + x.split('_')[0]},inplace = True)
replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
df_antzuola_pluviometro.replace('-1.0', np.nan, inplace=True)

Antzuola_Pluviometro_01oct2018_30sep2019.csv
Antzuola_Pluviometro_01oct2019_30sep2020.csv
Antzuola_Pluviometro_01oct2020_30sep2021.csv
Antzuola_Pluviometro_01oct2021_30sep2022.csv
Antzuola_Pluviometro_01oct2022_30sep2023.csv


In [12]:
urola_aux1, x = read_database(urola_raw_path, 83,109, 'Pluviometro')
df_arantzazu_pluviometro = df_fechas.copy()
df_arantzazu_pluviometro = pd.merge(df_arantzazu_pluviometro, urola_aux1, left_on='Fecha', right_on='FECHA', how='inner')
df_arantzazu_pluviometro = df_arantzazu_pluviometro.drop(columns=['FECHA'])
df_arantzazu_pluviometro.rename(columns={'LLUVIA (mm/m2)': 'LLUVIA_' + x.split('_')[0]},inplace = True)
replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
df_aitzu.replace(replace_dict, inplace=True)
df_arantzazu_pluviometro.replace('-1', np.nan, inplace=True)

Arantzazu_Pluviometro_01oct1997_30sep1998.csv
Arantzazu_Pluviometro_01oct1998_30sep1999.csv
Arantzazu_Pluviometro_01oct1999_30sep2000.csv
Arantzazu_Pluviometro_01oct2000_30sep2001.csv
Arantzazu_Pluviometro_01oct2001_30sep2002.csv
Arantzazu_Pluviometro_01oct2002_30sep2003.csv
Arantzazu_Pluviometro_01oct2003_30sep2004.csv
Arantzazu_Pluviometro_01oct2004_30sep2005.csv
Arantzazu_Pluviometro_01oct2005_30sep2006.csv
Arantzazu_Pluviometro_01oct2006_30sep2007.csv
Arantzazu_Pluviometro_01oct2007_30sep2008.csv
Arantzazu_Pluviometro_01oct2008_30sep2009.csv
Arantzazu_Pluviometro_01oct2009_30sep2010.csv
Arantzazu_Pluviometro_01oct2010_30sep2011.csv
Arantzazu_Pluviometro_01oct2011_30sep2012.csv
Arantzazu_Pluviometro_01oct2012_30sep2013.csv
Arantzazu_Pluviometro_01oct2013_30sep2014.csv
Arantzazu_Pluviometro_01oct2014_30sep2015.csv
Arantzazu_Pluviometro_01oct2015_30sep2016.csv
Arantzazu_Pluviometro_01oct2016_30sep2017.csv
Arantzazu_Pluviometro_01oct2017_30sep2018.csv
Arantzazu_Pluviometro_01oct2018_30

In [13]:
urola_aux1, x = read_database(urola_raw_path, 109,135, 'Pluviometro')    
df_araotz_pluviometro = df_fechas.copy()
df_araotz_pluviometro = pd.merge(df_araotz_pluviometro, urola_aux1, left_on='Fecha', right_on='FECHA', how='inner')
df_araotz_pluviometro = df_araotz_pluviometro.drop(columns=['FECHA'])
df_araotz_pluviometro.rename(columns={'LLUVIA (mm/m2)': 'LLUVIA_' + x.split('_')[0]},inplace = True)
replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
df_aitzu.replace(replace_dict, inplace=True)
df_araotz_pluviometro.replace('-1', np.nan, inplace=True)

Araotz_Pluviometro_01oct1990_30sep1991.csv
Araotz_Pluviometro_01oct1991_30sep1992.csv
Araotz_Pluviometro_01oct1992_30sep1993.csv
Araotz_Pluviometro_01oct1993_30sep1994.csv
Araotz_Pluviometro_01oct1994_30sep1995.csv
Araotz_Pluviometro_01oct1995_30sep1996.csv
Araotz_Pluviometro_01oct1996_30sep1997.csv
Araotz_Pluviometro_01oct2004_30sep2005.csv
Araotz_Pluviometro_01oct2005_30sep2006.csv
Araotz_Pluviometro_01oct2006_30sep2007.csv
Araotz_Pluviometro_01oct2007_30sep2008.csv
Araotz_Pluviometro_01oct2008_30sep2009.csv
Araotz_Pluviometro_01oct2009_30sep2010.csv
Araotz_Pluviometro_01oct2010_30sep2011.csv
Araotz_Pluviometro_01oct2011_30sep2012.csv
Araotz_Pluviometro_01oct2012_30sep2013.csv
Araotz_Pluviometro_01oct2013_30sep2014.csv
Araotz_Pluviometro_01oct2014_30sep2015.csv
Araotz_Pluviometro_01oct2015_30sep2016.csv
Araotz_Pluviometro_01oct2016_30sep2017.csv
Araotz_Pluviometro_01oct2017_30sep2018.csv
Araotz_Pluviometro_01oct2018_30sep2019.csv
Araotz_Pluviometro_01oct2019_30sep2020.csv
Araotz_Pluv

In [14]:
urola_aux1, x = read_database(urola_raw_path, 135,168, 'Pluviometro')
df_arkaka_pluviometro = df_fechas.copy()
df_arkaka_pluviometro = pd.merge(df_arkaka_pluviometro, urola_aux1, left_on='Fecha', right_on='FECHA', how='inner')
df_arkaka_pluviometro = df_arkaka_pluviometro.drop(columns=['FECHA'])
df_arkaka_pluviometro.rename(columns={'LLUVIA (mm/m2)': 'LLUVIA_' + x.split('_')[0]},inplace = True)
replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
df_arkaka_pluviometro.replace('-1', np.nan, inplace=True)

Arkaka_Pluviometro_01oct1990_30sep1991.csv
Arkaka_Pluviometro_01oct1991_30sep1992.csv
Arkaka_Pluviometro_01oct1992_30sep1993.csv
Arkaka_Pluviometro_01oct1993_30sep1994.csv
Arkaka_Pluviometro_01oct1994_30sep1995.csv
Arkaka_Pluviometro_01oct1995_30sep1996.csv
Arkaka_Pluviometro_01oct1996_30sep1997.csv
Arkaka_Pluviometro_01oct1997_30sep1998.csv
Arkaka_Pluviometro_01oct1998_30sep1999.csv
Arkaka_Pluviometro_01oct1999_30sep2000.csv
Arkaka_Pluviometro_01oct2000_30sep2001.csv
Arkaka_Pluviometro_01oct2001_30sep2002.csv
Arkaka_Pluviometro_01oct2002_30sep2003.csv
Arkaka_Pluviometro_01oct2003_30sep2004.csv
Arkaka_Pluviometro_01oct2004_30sep2005.csv
Arkaka_Pluviometro_01oct2005_30sep2006.csv
Arkaka_Pluviometro_01oct2006_30sep2007.csv
Arkaka_Pluviometro_01oct2007_30sep2008.csv
Arkaka_Pluviometro_01oct2008_30sep2009.csv
Arkaka_Pluviometro_01oct2009_30sep2010.csv
Arkaka_Pluviometro_01oct2010_30sep2011.csv
Arkaka_Pluviometro_01oct2011_30sep2012.csv
Arkaka_Pluviometro_01oct2012_30sep2013.csv
Arkaka_Pluv

In [15]:
urola_aux1, x = read_database(urola_raw_path, 200,233, 'Pluviometro')
df_Barrendiola_Pluviometro = df_fechas.copy()
df_Barrendiola_Pluviometro = pd.merge(df_Barrendiola_Pluviometro, urola_aux1, left_on='Fecha', right_on='FECHA', how='inner')
df_Barrendiola_Pluviometro = df_Barrendiola_Pluviometro.drop(columns=['FECHA'])
df_Barrendiola_Pluviometro.rename(columns={'LLUVIA (mm/m2)': 'LLUVIA_' + x.split('_')[0] + '_' + x.split('_')[1]},inplace = True)
replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
df_Barrendiola_Pluviometro.replace(replace_dict, inplace=True)


Barrendiola_Pluviometro_01oct1990_30sep1991.csv
Barrendiola_Pluviometro_01oct1991_30sep1992.csv
Barrendiola_Pluviometro_01oct1992_30sep1993.csv
Barrendiola_Pluviometro_01oct1993_30sep1994.csv
Barrendiola_Pluviometro_01oct1994_30sep1995.csv
Barrendiola_Pluviometro_01oct1995_30sep1996.csv
Barrendiola_Pluviometro_01oct1996_30sep1997.csv
Barrendiola_Pluviometro_01oct1997_30sep1998.csv
Barrendiola_Pluviometro_01oct1998_30sep1999.csv
Barrendiola_Pluviometro_01oct1999_30sep2000.csv
Barrendiola_Pluviometro_01oct2000_30sep2001.csv
Barrendiola_Pluviometro_01oct2001_30sep2002.csv
Barrendiola_Pluviometro_01oct2002_30sep2003.csv
Barrendiola_Pluviometro_01oct2003_30sep2004.csv
Barrendiola_Pluviometro_01oct2004_30sep2005.csv
Barrendiola_Pluviometro_01oct2005_30sep2006.csv
Barrendiola_Pluviometro_01oct2006_30sep2007.csv
Barrendiola_Pluviometro_01oct2007_30sep2008.csv
Barrendiola_Pluviometro_01oct2008_30sep2009.csv
Barrendiola_Pluviometro_01oct2009_30sep2010.csv
Barrendiola_Pluviometro_01oct2010_30sep2

In [16]:
urola_aux1, x = read_database(urola_raw_path, 233,257, 'Pluviometro')    
df_Elosua_Pluviometro = df_fechas.copy()
df_Elosua_Pluviometro = pd.merge(df_Elosua_Pluviometro, urola_aux1, left_on='Fecha', right_on='FECHA', how='inner')
df_Elosua_Pluviometro = df_Elosua_Pluviometro.drop(columns=['FECHA'])
df_Elosua_Pluviometro.rename(columns={'LLUVIA (mm/m2)': 'LLUVIA_' + x.split('_')[0]},inplace = True)
replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
df_Elosua_Pluviometro.replace(replace_dict, inplace=True)

Elosua_Pluviometro_01oct1999_30sep2000.csv
Elosua_Pluviometro_01oct2000_30sep2001.csv
Elosua_Pluviometro_01oct2001_30sep2002.csv
Elosua_Pluviometro_01oct2002_30sep2003.csv
Elosua_Pluviometro_01oct2003_30sep2004.csv
Elosua_Pluviometro_01oct2004_30sep2005.csv
Elosua_Pluviometro_01oct2005_30sep2006.csv
Elosua_Pluviometro_01oct2006_30sep2007.csv
Elosua_Pluviometro_01oct2007_30sep2008.csv
Elosua_Pluviometro_01oct2008_30sep2009.csv
Elosua_Pluviometro_01oct2009_30sep2010.csv
Elosua_Pluviometro_01oct2010_30sep2011.csv
Elosua_Pluviometro_01oct2011_30sep2012.csv
Elosua_Pluviometro_01oct2012_30sep2013.csv
Elosua_Pluviometro_01oct2013_30sep2014.csv
Elosua_Pluviometro_01oct2014_30sep2015.csv
Elosua_Pluviometro_01oct2015_30sep2016.csv
Elosua_Pluviometro_01oct2016_30sep2017.csv
Elosua_Pluviometro_01oct2017_30sep2018.csv
Elosua_Pluviometro_01oct2018_30sep2019.csv
Elosua_Pluviometro_01oct2019_30sep2020.csv
Elosua_Pluviometro_01oct2020_30sep2021.csv
Elosua_Pluviometro_01oct2021_30sep2022.csv
Elosua_Pluv

In [17]:
urola_aux1, x = read_database(urola_raw_path, 265,290, 'Pluviometro')    
df_Erdoizta_Pluviometro = df_fechas.copy()
df_Erdoizta_Pluviometro = pd.merge(df_Erdoizta_Pluviometro, urola_aux1, left_on='Fecha', right_on='FECHA', how='inner')
df_Erdoizta_Pluviometro = df_Erdoizta_Pluviometro.drop(columns=['FECHA'])
df_Erdoizta_Pluviometro.rename(columns={'LLUVIA (mm/m2)': 'LLUVIA_' + x.split('_')[0]},inplace = True)
replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
df_Erdoizta_Pluviometro.replace(replace_dict, inplace=True)
# df_Erdoizta_Pluviometro = df_Erdoizta_Pluviometro.groupby(df_Erdoizta_Pluviometro.index.Fecha).mean().reset_index()

Erdoizta_Pluviometro_01oct1998_30sep1999.csv
Erdoizta_Pluviometro_01oct1999_30sep2000.csv
Erdoizta_Pluviometro_01oct2000_30sep2001.csv
Erdoizta_Pluviometro_01oct2001_30sep2002.csv
Erdoizta_Pluviometro_01oct2002_30sep2003.csv
Erdoizta_Pluviometro_01oct2003_30sep2004.csv
Erdoizta_Pluviometro_01oct2004_30sep2005.csv
Erdoizta_Pluviometro_01oct2005_30sep2006.csv
Erdoizta_Pluviometro_01oct2006_30sep2007.csv
Erdoizta_Pluviometro_01oct2007_30sep2008.csv
Erdoizta_Pluviometro_01oct2008_30sep2009.csv
Erdoizta_Pluviometro_01oct2009_30sep2010.csv
Erdoizta_Pluviometro_01oct2010_30sep2011.csv
Erdoizta_Pluviometro_01oct2011_30sep2012.csv
Erdoizta_Pluviometro_01oct2012_30sep2013.csv
Erdoizta_Pluviometro_01oct2013_30sep2014.csv
Erdoizta_Pluviometro_01oct2014_30sep2015.csv
Erdoizta_Pluviometro_01oct2015_30sep2016.csv
Erdoizta_Pluviometro_01oct2016_30sep2017.csv
Erdoizta_Pluviometro_01oct2017_30sep2018.csv
Erdoizta_Pluviometro_01oct2018_30sep2019.csv
Erdoizta_Pluviometro_01oct2019_30sep2020.csv
Erdoizta_P

In [19]:
urola_aux1, x = read_database(urola_raw_path, 323,356, 'Pluviometro')  
df_Lareo_Pluviometro = df_fechas.copy()
df_Lareo_Pluviometro = pd.merge(df_Lareo_Pluviometro, urola_aux1, left_on='Fecha', right_on='FECHA', how='inner')
df_Lareo_Pluviometro = df_Lareo_Pluviometro.drop(columns=['FECHA'])
df_Lareo_Pluviometro.rename(columns={'LLUVIA (mm/m2)': 'LLUVIA_' + x.split('_')[0]},inplace = True)
replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
df_Lareo_Pluviometro.replace(replace_dict, inplace=True)

Lareo_Pluviometro_01oct1990_30sep1991.csv
Lareo_Pluviometro_01oct1991_30sep1992.csv
Lareo_Pluviometro_01oct1992_30sep1993.csv
Lareo_Pluviometro_01oct1993_30sep1994.csv
Lareo_Pluviometro_01oct1994_30sep1995.csv
Lareo_Pluviometro_01oct1995_30sep1996.csv
Lareo_Pluviometro_01oct1996_30sep1997.csv
Lareo_Pluviometro_01oct1997_30sep1998.csv
Lareo_Pluviometro_01oct1998_30sep1999.csv
Lareo_Pluviometro_01oct1999_30sep2000.csv
Lareo_Pluviometro_01oct2000_30sep2001.csv
Lareo_Pluviometro_01oct2001_30sep2002.csv
Lareo_Pluviometro_01oct2002_30sep2003.csv
Lareo_Pluviometro_01oct2003_30sep2004.csv
Lareo_Pluviometro_01oct2004_30sep2005.csv
Lareo_Pluviometro_01oct2005_30sep2006.csv
Lareo_Pluviometro_01oct2006_30sep2007.csv
Lareo_Pluviometro_01oct2007_30sep2008.csv
Lareo_Pluviometro_01oct2008_30sep2009.csv
Lareo_Pluviometro_01oct2009_30sep2010.csv
Lareo_Pluviometro_01oct2010_30sep2011.csv
Lareo_Pluviometro_01oct2011_30sep2012.csv
Lareo_Pluviometro_01oct2012_30sep2013.csv
Lareo_Pluviometro_01oct2013_30sep2

In [20]:
urola_aux1, x = read_database(urola_raw_path, 356,389, 'Pluviometro')    
df_Lastur_Pluviometro = df_fechas.copy()
df_Lastur_Pluviometro = pd.merge(df_Lastur_Pluviometro, urola_aux1, left_on='Fecha', right_on='FECHA', how='inner')
df_Lastur_Pluviometro = df_Lastur_Pluviometro.drop(columns=['FECHA'])
df_Lastur_Pluviometro.rename(columns={'LLUVIA (mm/m2)': 'LLUVIA_' + x.split('_')[0]},inplace = True)
replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
df_Lastur_Pluviometro.replace(replace_dict, inplace=True)

Lastur_Pluviometro_01oct1990_30sep1991.csv
Lastur_Pluviometro_01oct1991_30sep1992.csv
Lastur_Pluviometro_01oct1992_30sep1993.csv
Lastur_Pluviometro_01oct1993_30sep1994.csv
Lastur_Pluviometro_01oct1994_30sep1995.csv
Lastur_Pluviometro_01oct1995_30sep1996.csv
Lastur_Pluviometro_01oct1996_30sep1997.csv
Lastur_Pluviometro_01oct1997_30sep1998.csv
Lastur_Pluviometro_01oct1998_30sep1999.csv
Lastur_Pluviometro_01oct1999_30sep2000.csv
Lastur_Pluviometro_01oct2000_30sep2001.csv
Lastur_Pluviometro_01oct2001_30sep2002.csv
Lastur_Pluviometro_01oct2002_30sep2003.csv
Lastur_Pluviometro_01oct2003_30sep2004.csv
Lastur_Pluviometro_01oct2004_30sep2005.csv
Lastur_Pluviometro_01oct2005_30sep2006.csv
Lastur_Pluviometro_01oct2006_30sep2007.csv
Lastur_Pluviometro_01oct2007_30sep2008.csv
Lastur_Pluviometro_01oct2008_30sep2009.csv
Lastur_Pluviometro_01oct2009_30sep2010.csv
Lastur_Pluviometro_01oct2010_30sep2011.csv
Lastur_Pluviometro_01oct2011_30sep2012.csv
Lastur_Pluviometro_01oct2012_30sep2013.csv
Lastur_Pluv

In [21]:
urola_aux1, x = read_database(urola_raw_path, 389,422, 'Pluviometro') 
df_Laurgain_Pluviometro = df_fechas.copy()
df_Laurgain_Pluviometro = pd.merge(df_Laurgain_Pluviometro, urola_aux1, left_on='Fecha', right_on='FECHA', how='inner')
df_Laurgain_Pluviometro = df_Laurgain_Pluviometro.drop(columns=['FECHA'])
df_Laurgain_Pluviometro.rename(columns={'LLUVIA (mm/m2)': 'LLUVIA_' + x.split('_')[0]},inplace = True)
replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
df_Laurgain_Pluviometro.replace(replace_dict, inplace=True)

Laurgain_Pluviometro_01oct1990_30sep1991.csv
Laurgain_Pluviometro_01oct1991_30sep1992.csv
Laurgain_Pluviometro_01oct1992_30sep1993.csv
Laurgain_Pluviometro_01oct1993_30sep1994.csv
Laurgain_Pluviometro_01oct1994_30sep1995.csv
Laurgain_Pluviometro_01oct1995_30sep1996.csv
Laurgain_Pluviometro_01oct1996_30sep1997.csv
Laurgain_Pluviometro_01oct1997_30sep1998.csv
Laurgain_Pluviometro_01oct1998_30sep1999.csv
Laurgain_Pluviometro_01oct1999_30sep2000.csv
Laurgain_Pluviometro_01oct2000_30sep2001.csv
Laurgain_Pluviometro_01oct2001_30sep2002.csv
Laurgain_Pluviometro_01oct2002_30sep2003.csv
Laurgain_Pluviometro_01oct2003_30sep2004.csv
Laurgain_Pluviometro_01oct2004_30sep2005.csv
Laurgain_Pluviometro_01oct2005_30sep2006.csv
Laurgain_Pluviometro_01oct2006_30sep2007.csv
Laurgain_Pluviometro_01oct2007_30sep2008.csv
Laurgain_Pluviometro_01oct2008_30sep2009.csv
Laurgain_Pluviometro_01oct2009_30sep2010.csv
Laurgain_Pluviometro_01oct2010_30sep2011.csv
Laurgain_Pluviometro_01oct2011_30sep2012.csv
Laurgain_P

In [22]:
urola_aux1, x = read_database(urola_raw_path, 444,474, 'Pluviometro')
df_Troya_Pluviometro = df_fechas.copy()
df_Troya_Pluviometro = pd.merge(df_Troya_Pluviometro, urola_aux1, left_on='Fecha', right_on='FECHA', how='inner')
df_Troya_Pluviometro = df_Troya_Pluviometro.drop(columns=['FECHA'])
df_Troya_Pluviometro.rename(columns={'LLUVIA (mm/m2)': 'LLUVIA_' + x.split('_')[0]},inplace = True)
replace_dict = {-1: np.nan, -999.0: np.nan, -99.9: np.nan, -999: np.nan, 99.0: np.nan, 999.9: np.nan, 99: np.nan}
df_Troya_Pluviometro.replace(replace_dict, inplace=True)

Troya_Pluviometro_01oct1993_30sep1994.csv
Troya_Pluviometro_01oct1994_30sep1995.csv
Troya_Pluviometro_01oct1995_30sep1996.csv
Troya_Pluviometro_01oct1996_30sep1997.csv
Troya_Pluviometro_01oct1997_30sep1998.csv
Troya_Pluviometro_01oct1998_30sep1999.csv
Troya_Pluviometro_01oct1999_30sep2000.csv
Troya_Pluviometro_01oct2000_30sep2001.csv
Troya_Pluviometro_01oct2001_30sep2002.csv
Troya_Pluviometro_01oct2002_30sep2003.csv
Troya_Pluviometro_01oct2003_30sep2004.csv
Troya_Pluviometro_01oct2004_30sep2005.csv
Troya_Pluviometro_01oct2005_30sep2006.csv
Troya_Pluviometro_01oct2006_30sep2007.csv
Troya_Pluviometro_01oct2007_30sep2008.csv
Troya_Pluviometro_01oct2008_30sep2009.csv
Troya_Pluviometro_01oct2009_30sep2010.csv
Troya_Pluviometro_01oct2010_30sep2011.csv
Troya_Pluviometro_01oct2011_30sep2012.csv
Troya_Pluviometro_01oct2012_30sep2013.csv
Troya_Pluviometro_01oct2013_30sep2014.csv
Troya_Pluviometro_01oct2014_30sep2015.csv
Troya_Pluviometro_01oct2015_30sep2016.csv
Troya_Pluviometro_01oct2016_30sep2

In [23]:
# Informacion sobre la base de datos: periodo 2009-2023 diezminutales
azpeitia = pd.read_csv(urola_raw_path + 'azpeitia.txt', sep=';', encoding='latin1')
#asignar nombres a las columnas
azpeitia.columns = ['FECHA', 'LLUVIA_azpeitia', 'VIENTO1', 'VIENTO2']
azpeitia['FECHA'] = azpeitia['FECHA'].astype(str).str.zfill(10)
azpeitia['FECHA']  =  pd.to_datetime(azpeitia['FECHA'], format='%y%m%d%H%M', errors='coerce')
# convertir la columna fecha a datetime y asignarla como indice
azpeitia['FECHA'] = pd.to_datetime(azpeitia['FECHA'])
azpeitia.set_index('FECHA', inplace=True)
azpeitia = azpeitia.drop(columns=['VIENTO1', 'VIENTO2'])

df_azpeitia = df_fechas.copy()
df_azpeitia = pd.merge(df_azpeitia, azpeitia, left_on='Fecha', right_on='FECHA', how='inner')

# Informacion sobre la base de datos: periodo 2009-2023 diezminutales
zumarraga = pd.read_csv(urola_raw_path + 'zumarraga.txt', sep=';', encoding='latin1')
#asignar nombres a las columnas
zumarraga.columns = ['FECHA', 'LLUVIA_zumarraga', 'VIENTO1', 'VIENTO2']
zumarraga['FECHA'] = zumarraga['FECHA'].astype(str).str.zfill(10)
zumarraga['FECHA']  =  pd.to_datetime(zumarraga['FECHA'], format='%y%m%d%H%M', errors='coerce')
# convertir la columna fecha a datetime y asignarla como indice
zumarraga['FECHA'] = pd.to_datetime(zumarraga['FECHA'])
zumarraga.set_index('FECHA', inplace=True)
zumarraga = zumarraga.drop(columns=['VIENTO1', 'VIENTO2'])

df_zumarraga = df_fechas.copy()
df_zumarraga = pd.merge(df_zumarraga, zumarraga, left_on='Fecha', right_on='FECHA', how='inner')

# Informacion sobre la base de datos: periodo 2009-2023 diezminutales
legazpi = pd.read_csv(urola_raw_path + 'legazpi.txt', sep=';', encoding='latin1')
#asignar nombres a las columnas
legazpi.columns = ['FECHA', 'LLUVIA_legazpi']
legazpi['FECHA'] = legazpi['FECHA'].astype(str).str.zfill(10)
legazpi['FECHA']  =  pd.to_datetime(legazpi['FECHA'], format='%y%m%d%H%M', errors='coerce')
# convertir la columna fecha a datetime y asignarla como indice
legazpi['FECHA'] = pd.to_datetime(legazpi['FECHA'])
legazpi.set_index('FECHA', inplace=True)

df_legazpi = df_fechas.copy()
df_legazpi = pd.merge(df_legazpi, legazpi, left_on='Fecha', right_on='FECHA', how='inner')



In [24]:
db_1 = pd.merge(df_fechas, df_aitzu_urola, on='Fecha', how='outer')
db_2 = pd.merge(db_1, df_aitzu, on='Fecha', how='outer')
db_3 = pd.merge(db_2, df_aizarnazabal, on='Fecha', how='outer')
db_4 = pd.merge(db_3, df_antzuola_pluviometro, on='Fecha', how='outer')
db_5 = pd.merge(db_4, df_arantzazu_pluviometro, on='Fecha', how='outer')
db_6 = pd.merge(db_5, df_araotz_pluviometro, on='Fecha', how='outer')
db_7 = pd.merge(db_6, df_arkaka_pluviometro, on='Fecha', how='outer')
db_8 = pd.merge(db_7, df_barrendiola_estacion, on='Fecha', how='outer')
db_9 = pd.merge(db_8, df_Barrendiola_Pluviometro, on='Fecha', how='outer')
db_10 = pd.merge(db_9, df_Elosua_Pluviometro, on='Fecha', how='outer')
# db_10 = db_10.replace(-1, np.nan)

db_10.to_csv('db_10.csv', index=False)

print(db_10.max())
print(db_10.min())

Fecha                             2024-12-31 23:00:00
LLUVIA_Aitzu-Urola                               11.1
NIVEL_RIO_Aitzu-Urola                           1.798
CAUDAL_RIO_Aitzu-Urola                        88.4015
LLUVIA_Aitzu                                     11.1
NIVEL_RIO_Aitzu                                 1.798
CAUDAL_RIO_Aitzu                              88.4015
LLUVIA_Aizarnazabal                              11.2
NIVEL_RIO_Aizarnazabal                          3.696
CAUDAL_RIO_Aizarnazabal                      459.5985
LLUVIA_Antzuola                                  11.5
LLUVIA_Arantzazu                                 11.6
LLUVIA_Araotz                                    14.9
LLUVIA_Arkaka                                    12.4
LLUVIA_Barrendiola                                8.2
NIVEL_RIO_Barrendiola                           1.176
CAUDAL_RIO_Barrendiola                         1.9549
LLUVIA_Barrendiola_Pluviometro                   10.6
LLUVIA_Elosua               

In [25]:
db_11 = pd.merge(db_10, df_Erdoizta_Pluviometro, on='Fecha', how='outer')
db_12 = pd.merge(db_11, df_Ibaieder_Estacion, on='Fecha', how='outer')
db_13 = pd.merge(db_12, df_Lareo_Pluviometro, on='Fecha', how='outer')
db_14 = pd.merge(db_13, df_Lastur_Pluviometro, on='Fecha', how='outer')
db_15 = pd.merge(db_14, df_Laurgain_Pluviometro, on='Fecha', how='outer')
db_16 = pd.merge(db_15, df_Matxinbenta_Estacion, on='Fecha', how='outer')
db_17 = pd.merge(db_16, df_Troya_Pluviometro, on='Fecha', how='outer')
db_18 = pd.merge(db_17, df_zumarraga, on='Fecha', how='outer')
db_19 = pd.merge(db_18, df_azpeitia, on='Fecha', how='outer')
db_20 = pd.merge(db_19, df_legazpi, on='Fecha', how='outer')
db_21 = db_20.replace(-1, np.nan)
db_21.drop_duplicates(subset='Fecha', keep='first', inplace=True)
db_21.to_csv('db_21.csv', index=False)
print(db_21.max())
print(db_21.min())

Fecha                             2024-12-31 23:00:00
LLUVIA_Aitzu-Urola                               11.1
NIVEL_RIO_Aitzu-Urola                           1.798
CAUDAL_RIO_Aitzu-Urola                        88.4015
LLUVIA_Aitzu                                     11.1
NIVEL_RIO_Aitzu                                 1.798
CAUDAL_RIO_Aitzu                              88.4015
LLUVIA_Aizarnazabal                              11.2
NIVEL_RIO_Aizarnazabal                          3.696
CAUDAL_RIO_Aizarnazabal                      459.5985
LLUVIA_Antzuola                                  11.5
LLUVIA_Arantzazu                                 11.6
LLUVIA_Araotz                                    14.9
LLUVIA_Arkaka                                    12.4
LLUVIA_Barrendiola                                8.2
NIVEL_RIO_Barrendiola                           1.176
CAUDAL_RIO_Barrendiola                         1.9549
LLUVIA_Barrendiola_Pluviometro                   10.6
LLUVIA_Elosua               

In [26]:
fecha_inicio = '2008-11-21 00:00'
fecha_fin = '2023-12-31 00:00'
fecha_fin = pd.to_datetime(fecha_fin)
# Ver datos desde las fechas de inicio y fin
rango_fechas = db_21[(db_21['Fecha'] >= fecha_inicio) & (db_21['Fecha'] <= fecha_fin)] 
rango_fechas

,Fecha,LLUVIA_Aitzu-Urola,NIVEL_RIO_Aitzu-Urola,CAUDAL_RIO_Aitzu-Urola,LLUVIA_Aitzu,NIVEL_RIO_Aitzu,CAUDAL_RIO_Aitzu,LLUVIA_Aizarnazabal,NIVEL_RIO_Aizarnazabal,CAUDAL_RIO_Aizarnazabal,...,LLUVIA_Lareo,LLUVIA_Lastur,LLUVIA_Laurgain,LLUVIA_Matxinbenta,NIVEL_RIO_Matxinbenta,CAUDAL_RIO_Matxinbenta,LLUVIA_Troya,LLUVIA_zumarraga,LLUVIA_azpeitia,LLUVIA_legazpi
95448,2008-11-21 00:00:00,0.0,0.404,3.4027,0.0,0.404,3.4027,0.0,0.790,17.6210,...,0.0,0.0,0.0,0.0,0.251,0.4863,0.0,NaN,NaN,0.0
95450,2008-11-21 01:00:00,0.0,0.400,3.3164,0.0,0.400,3.3164,0.0,0.788,17.5079,...,0.0,0.0,0.0,0.0,0.250,0.4814,0.0,NaN,NaN,0.0
95451,2008-11-21 02:00:00,0.0,0.397,3.2524,0.0,0.397,3.2524,0.0,0.778,16.9487,...,0.0,0.0,0.0,0.0,0.249,0.4766,0.0,NaN,NaN,0.0
95452,2008-11-21 03:00:00,0.0,0.396,3.2312,0.0,0.396,3.2312,0.0,0.776,16.8381,...,0.0,0.0,0.0,0.0,0.247,0.4670,0.0,NaN,NaN,0.2
95453,2008-11-21 04:00:00,0.0,0.394,3.1890,0.0,0.394,3.1890,0.0,0.770,16.5086,...,0.2,0.0,0.0,0.0,0.246,0.4622,0.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264092,2023-12-30 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
264093,2023-12-30 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
264094,2023-12-30 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
264095,2023-12-30 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
